In [1]:
# Window
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { width:90% !important; }</style>"))
display(HTML(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """))

# Bounds – Integrated model for routing pollsters and vehicles

<div class="alert alert-block alert-info">
This notebook contains the iterative code to determine bounds on a set of instances for the Integrated Vehicles and Polsters Routing Problem (IVPRP).
</div>

In [1]:
# Packages
import numpy  as np
import pandas as pd
import time
from collections import deque

In [2]:
# Aliases
append, arange, around, asarray, loadtxt, zeros = np.append, np.arange, np.around, np.asarray, np.loadtxt, np.zeros
DataFrame, concat = pd.DataFrame, pd.concat

In [3]:
# General functions for reading and writing
def Reader(nom):    return loadtxt('Instances/'+ nom + '.txt' )
def P_Reader(n):    return  Reader('Pollster_' + str(n))
def V_Reader(n):    return  Reader( 'Vehicle_' + str(n))
def S_Reader(n):    return  Reader( 'Service_' + str(n))
def T_Reader(n):    return  Reader(    'Time_' + str(n))

def Write(df,nom):  return DataFrame(df).to_csv('Instances/'+ nom +'.txt', header=None, index=None, sep=' ', mode='a')

In [4]:
print('NumPy: {0}, Pandas: {1}'.format(np.__version__, pd.__version__))

NumPy: 1.26.4, Pandas: 2.2.3


## Basics

<div class="alert alert-block alert-warning">
A general list is needed to call and run all the IP's at once. This list will store:
    
* $n$ is the number of stores to be visited,
* $E$ is the set of available pollsters,
* $K$ is the set of available vehicles,
* $Q$ is the vehicle's capacity,
* $S$ is the number of days.
</div>

In [5]:
#      (n, K, E, S, Q)
Γ = ( (10, 3, 3, 1, 1), (12, 2, 2, 2, 1), (14, 2, 2, 2, 1), (16, 2, 3, 2, 2), (18, 2, 3, 2, 2), 
      (20, 3, 4, 1, 2), (25, 2, 2, 2, 3), (30, 2, 2, 2, 3), (40, 2, 4, 2, 4), (50, 3, 5, 2, 4),
      (60, 3, 5, 2, 4), (70, 3, 5, 3, 4), (80, 3, 5, 3, 4), (90, 3, 5, 3, 4), (100, 3, 5, 4, 4)
    )

# Iterative code

Now we iterate over the set of instances and compute lower and upper bounds based on the expressions from the paper.

<div class="alert alert-block alert-warning">
At last costs are fixed in advance, as they are not instance dependent:
    
* $\kappa_0$ is the daily cost of operations,
* $\kappa_1$ is the cost of hiring a vehicle,
* $\kappa_2$ is the cost of hiring a pollster.
</div>

In [6]:
κ_0 = 200.0
κ_1 = 100.0
κ_2 = 40.0

<div class="alert alert-block alert-success">
Using the generator created above, we build the basic index sets to build the model and read data.
</div>
<div class="alert alert-block alert-warning">
The read matrices are processed:
    
* $d$ captures pollstering times,
* $t$ is the time that a pollster takes to walk among pairs of stores,
* $\tau$ is the time that vehicles take to move between pairs of stores,

* $[\rho_0,\rho_1]$ is the time window for breaks,
* $P$ is the length of the pause,
* $\beta$ is the time horizon limit.
</div>
</div>

## Upper bounds
\begin{align}
    &\min_{m} \quad \kappa_0 m_S + \kappa_1 m_K m_S + \kappa_{2} m_E m_S
    \\
    \text{subject to} \qquad &
    \\
    & P + \dfrac{ \sum_{i\in C_-} t_{i,i+n} }{m_E m_S} + \dfrac{ \left( \sum \max_{i} \tau_{} + \max_{j} \tau \right) }{ m_K m_S } \leq B_{\max},
    \\
    & m_K \leq m_E \leq Q m_K,
    \\
    & m = (m_S, m_K, m_E) \in \big[1:|S|\big] \times \big[1:|K|\big] \times \big[1:|E|\big].
\end{align}

## Lower bounds
\begin{align}
    \ell_E &= \operatorname{arg max} \left\{ \frac{1}{\ell_e} \sum_{i\in C_-} t_{i,i+n} : \, \sum_{i\in C_-} t_{i,i+n} \leq (B_{\max} - P) \, \ell_e \wedge \ell_e \in \big[1:|S||E|\big] \right\},
    \\
    \ell_S &= \operatorname{arg min}  \left\{ \ell_E \in \big[s:s |E| \big] : \, s \in \big[1:|S| \big] \right\},
    \\
    \ell_K &= \min \left\{ \ell_k : \, \ell_E \leq Q \ell_k \wedge \ell_k \in \big[\ell_S:\ell_S |K|\big] \right\}.
\end{align}

Now we find the bounds. For each quantity, we present the values $[\ell,m]$ next to the given parameter of the instance. For example, the vehicle information will display $[\ell_K, m_K](K)$.

In [7]:
print('{0:^8}|{1:^10}|{2:^11}|{3:^10}|{4:^10}'.format('Stores','Vehicles','Pollsters','Days','Capacity'))
print('{0:^8}|{1:^10}|{2:^11}|{3:^10}|{4:^10}'.format('n','K','E','S','Q'))
print('–'*53)
for γ in Γ:
    n = γ[0];    E = arange(γ[2]);    K = arange(γ[1]);    Q = γ[-1];    S = arange(γ[3])
    #print('*** Bounds for instance of size:', n, ' ***\n')
    
    '''Reading data'''
    Service  = S_Reader(n)
    Poll     = P_Reader(n)
    Vehicles = V_Reader(n)
    Time     = T_Reader(n)
    
    '''Processing data'''
    d = append(around(Service, 2), 0.0)
    t = around(Poll, 2)
    τ = around(Vehicles, 2)
    ρ_0, ρ_1, P, β = Time.ravel()
    
    # Upper bounds
    Feas = {(e+1,k+1,s+1): (d.sum() + (e+1)*(s+1)*P)/((e+1)*(s+1)) 
                + (τ.max(axis=0) + τ.max(axis=1)).sum()/((k+1)*(s+1)*(1)) for e in E for k in K for s in S}
    Feas = {aa: κ_2 * aa[0] * aa[2] + κ_1 * aa[1] * aa[2] + κ_0 * aa[2] for aa,bb in Feas.items() if (bb < β) and (aa[1] <= aa[0] <= Q * (aa[1]))}
    m_E, m_K, m_S = min(Feas, key=Feas.get)
    

    # Lower bounds
    Λ   = [ np.append(a, arange(a.size + b + 1, (b+1)*a.size + 1) ) for b,a in enumerate([(E+1)+s for s in S]) ]
    Λ_β = {λ: d.sum()/λ for λ in np.unique(np.concatenate(Λ)) if d.sum()/λ <= β - P}

    ℓ_E, T_1 = max(Λ_β.items(), key=lambda x: x[1]) 
    ℓ_S = next(a for a,b in enumerate(Λ) if ℓ_E in b) + 1
    ℓ_K = [k for k in arange(ℓ_S, K.size * ℓ_S + 1) if ℓ_E <= Q * k].pop(0)
    #print('Lower bounds found: |S| >=', ℓ_S, '|K| >=',ℓ_K, '|E| >=', ℓ_E)
    
    #print('\n\n')
    print('{0:^8}|{1:^10}|{2:^11}|{3:^10}|{4:^10}'.format(n,  '[{0:1d},{1:1d}]({2})'.format(ℓ_K, m_K, γ[1]),  
                                             '[{0:1d},{1:1d}]({2})'.format(ℓ_E, m_E, γ[2]),  
                                             '[{0:1d},{1:1d}]({2})'.format(ℓ_S, m_S, γ[3]),   γ[-1] ))
    
print('–'*53)

 Stores | Vehicles | Pollsters |   Days   | Capacity 
   n    |    K     |     E     |    S     |    Q     
–––––––––––––––––––––––––––––––––––––––––––––––––––––
   10   | [2,3](3) | [2,3](3)  | [1,1](1) |    1     
   12   | [2,2](2) | [2,2](2)  | [1,2](2) |    1     
   14   | [2,2](2) | [2,2](2)  | [1,2](2) |    1     
   16   | [1,2](2) | [2,3](3)  | [1,2](2) |    2     
   18   | [1,2](2) | [2,3](3)  | [1,2](2) |    2     
   20   | [1,3](3) | [2,4](4)  | [1,1](1) |    2     
   25   | [1,2](2) | [2,2](2)  | [1,2](2) |    3     
   30   | [1,2](2) | [2,2](2)  | [1,2](2) |    3     
   40   | [1,2](2) | [3,4](4)  | [1,2](2) |    4     
   50   | [2,3](3) | [6,5](5)  | [2,2](2) |    4     
   60   | [1,2](3) | [3,3](5)  | [1,2](2) |    4     
   70   | [1,2](3) | [3,3](5)  | [1,2](3) |    4     
   80   | [1,2](3) | [4,4](5)  | [1,2](3) |    4     
   90   | [1,2](3) | [4,4](5)  | [1,2](3) |    4     
  100   | [1,2](3) | [4,5](5)  | [1,2](4) |    4     
––––––––––––––––––––––––––––

Notice that we do not necessarily have that $\ell_E \leq m_E$. This is because the value of $\ell_E$ bounds the cummulative number of pollsters across the time span. Instead, $m_E$ is independent of the number of days.

---